In [1]:
import cv2
import numpy as np

def get_color(color_name):
    """Converte nomes de cores simples para tuplas BGR do OpenCV."""
    mapping = {
        "black": (0, 0, 0),
        "white": (255, 255, 255),
        "green": (0, 255, 0),
        "blue": (255, 0, 0),
        # Acrescente outras cores se necessário
    }
    return mapping.get(color_name.lower(), (0, 0, 0))

def to_pixel(x, y, scale, offset_x, offset_y):
    """Converte coordenadas do campo (unidades) para coordenadas em pixels."""
    return (int((x + offset_x) * scale), int((y + offset_y) * scale))

def draw_statsbomb_pitch_cv(pitch_length=120, pitch_width=80,
                            origin_x=0, origin_y=0,
                            line_color="black", pitch_color="white",
                            linewidth=1, scale=5):

    goal_offset = 2
    x_min = origin_x - goal_offset - 5
    x_max = origin_x + pitch_length + goal_offset + 5
    y_min = origin_y - 5
    y_max = origin_y + pitch_width + 5
    
    width_px = int((x_max - x_min) * scale)
    height_px = int((y_max - y_min) * scale)
    
    # Offsets para converter as coordenadas do campo para a imagem
    offset_x = -x_min   # Ex: se x_min = -7, offset_x = 7
    offset_y = -y_min   # Ex: se y_min = -5, offset_y = 5
    
    # Cria imagem com fundo (ex.: cinza claro) para visualizar as margens
    img = np.full((height_px, width_px, 3), 200, dtype=np.uint8)
    
    # Converte cores para BGR
    line_bgr = get_color(line_color)
    pitch_bgr = get_color(pitch_color)
    
    # ------------------
    # 1. Retângulo principal (campo)
    # ------------------
    pt1 = to_pixel(origin_x, origin_y, scale, offset_x, offset_y)
    pt2 = to_pixel(origin_x + pitch_length, origin_y + pitch_width, scale, offset_x, offset_y)
    # Preenche o retângulo com a cor do campo
    cv2.rectangle(img, pt1, pt2, pitch_bgr, thickness=-1)
    # Desenha a borda
    cv2.rectangle(img, pt1, pt2, line_bgr, thickness=linewidth)
    
    # ------------------
    # 2. Linha central
    # ------------------
    mid_x = origin_x + pitch_length / 2
    pt_top = to_pixel(mid_x, origin_y, scale, offset_x, offset_y)
    pt_bot = to_pixel(mid_x, origin_y + pitch_width, scale, offset_x, offset_y)
    cv2.line(img, pt_top, pt_bot, line_bgr, thickness=linewidth, lineType=cv2.LINE_AA)
    
    # ------------------
    # 3. Círculo central e ponto central
    # ------------------
    center_field = (origin_x + pitch_length / 2, origin_y + pitch_width / 2)
    center_px = to_pixel(center_field[0], center_field[1], scale, offset_x, offset_y)
    radius_circle = int(10 * scale)
    cv2.circle(img, center_px, radius_circle, line_bgr, thickness=linewidth, lineType=cv2.LINE_AA)
    # Ponto central (círculo preenchido)
    cv2.circle(img, center_px, 2, line_bgr, thickness=-1, lineType=cv2.LINE_AA)
    
    # ------------------
    # 4. Áreas de pênalti (18 x 44)
    # ------------------
    left_penalty_origin = (origin_x, origin_y + (pitch_width - 44) / 2)
    left_penalty_pt1 = to_pixel(left_penalty_origin[0], left_penalty_origin[1], scale, offset_x, offset_y)
    left_penalty_pt2 = to_pixel(left_penalty_origin[0] + 18, left_penalty_origin[1] + 44, scale, offset_x, offset_y)
    cv2.rectangle(img, left_penalty_pt1, left_penalty_pt2, line_bgr, thickness=linewidth)
    
    right_penalty_origin = (origin_x + pitch_length - 18, origin_y + (pitch_width - 44) / 2)
    right_penalty_pt1 = to_pixel(right_penalty_origin[0], right_penalty_origin[1], scale, offset_x, offset_y)
    right_penalty_pt2 = to_pixel(right_penalty_origin[0] + 18, right_penalty_origin[1] + 44, scale, offset_x, offset_y)
    cv2.rectangle(img, right_penalty_pt1, right_penalty_pt2, line_bgr, thickness=linewidth)
    
    # ------------------
    # 5. Áreas dos 6 metros (6 x 20)
    # ------------------
    left_six_origin = (origin_x, origin_y + (pitch_width - 20) / 2)
    left_six_pt1 = to_pixel(left_six_origin[0], left_six_origin[1], scale, offset_x, offset_y)
    left_six_pt2 = to_pixel(left_six_origin[0] + 6, left_six_origin[1] + 20, scale, offset_x, offset_y)
    cv2.rectangle(img, left_six_pt1, left_six_pt2, line_bgr, thickness=linewidth)
    
    right_six_origin = (origin_x + pitch_length - 6, origin_y + (pitch_width - 20) / 2)
    right_six_pt1 = to_pixel(right_six_origin[0], right_six_origin[1], scale, offset_x, offset_y)
    right_six_pt2 = to_pixel(right_six_origin[0] + 6, right_six_origin[1] + 20, scale, offset_x, offset_y)
    cv2.rectangle(img, right_six_pt1, right_six_pt2, line_bgr, thickness=linewidth)
    
    # ------------------
    # 6. Pontos de pênalti (desenhados como círculos)
    # ------------------
    left_penalty_spot = (origin_x + 12, origin_y + pitch_width / 2)
    left_penalty_px = to_pixel(left_penalty_spot[0], left_penalty_spot[1], scale, offset_x, offset_y)
    cv2.circle(img, left_penalty_px, 2, line_bgr, thickness=-1, lineType=cv2.LINE_AA)
    
    right_penalty_spot = (origin_x + pitch_length - 12, origin_y + pitch_width / 2)
    right_penalty_px = to_pixel(right_penalty_spot[0], right_penalty_spot[1], scale, offset_x, offset_y)
    cv2.circle(img, right_penalty_px, 2, line_bgr, thickness=-1, lineType=cv2.LINE_AA)
    
    # ------------------
    # 7. Arcos de pênalti
    # ------------------
    # Arco esquerdo (de -53° a 53°; dividimos em duas partes para lidar com ângulos negativos)
    left_arc_center = left_penalty_spot  # mesma que o ponto de pênalti esquerdo
    left_arc_px = to_pixel(left_arc_center[0], left_arc_center[1], scale, offset_x, offset_y)
    axes = (int(10 * scale), int(10 * scale))  # raios em pixels
    
    # Desenha de 307° a 360° (equivalente a -53° a 0°)
    cv2.ellipse(img, left_arc_px, axes, 0, 307, 360, line_bgr, thickness=linewidth, lineType=cv2.LINE_AA)
    # Desenha de 0° a 53°
    cv2.ellipse(img, left_arc_px, axes, 0, 0, 53, line_bgr, thickness=linewidth, lineType=cv2.LINE_AA)
    
    # Arco direito: de 127° a 233°
    right_arc_center = right_penalty_spot
    right_arc_px = to_pixel(right_arc_center[0], right_arc_center[1], scale, offset_x, offset_y)
    cv2.ellipse(img, right_arc_px, axes, 0, 127, 233, line_bgr, thickness=linewidth, lineType=cv2.LINE_AA)
    
    # ------------------
    # 8. Gols
    # ------------------
    goal_width = 8
    left_goal_origin = (origin_x - goal_offset, origin_y + pitch_width / 2 - goal_width / 2)
    left_goal_pt1 = to_pixel(left_goal_origin[0], left_goal_origin[1], scale, offset_x, offset_y)
    left_goal_pt2 = to_pixel(left_goal_origin[0] + goal_offset, left_goal_origin[1] + goal_width, scale, offset_x, offset_y)
    cv2.rectangle(img, left_goal_pt1, left_goal_pt2, line_bgr, thickness=linewidth)
    
    right_goal_origin = (origin_x + pitch_length, origin_y + pitch_width / 2 - goal_width / 2)
    right_goal_pt1 = to_pixel(right_goal_origin[0], right_goal_origin[1], scale, offset_x, offset_y)
    right_goal_pt2 = to_pixel(right_goal_origin[0] + goal_offset, right_goal_origin[1] + goal_width, scale, offset_x, offset_y)
    cv2.rectangle(img, right_goal_pt1, right_goal_pt2, line_bgr, thickness=linewidth)
    
    return img

if __name__ == "__main__":
    img = draw_statsbomb_pitch_cv(scale=5, linewidth=1)
    cv2.imshow("Pitch StatsBomb (OpenCV)", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()